# Sequential

本文将展示Langchain结合qianfan使用`Sequential` 以及 `LCEL` 进行大模型应用Prompt和Chain的组装调用，原文请参考[Langchain Seuqential](https://python.langchain.com/docs/expression_language/)
总的来说Langchain更推荐是`LCEL`的方式进行实现

In [ ]:
#-# cell_skip
! pip install "qianfan>=0.2.2" -U
! pip install langchain 

In [ ]:
import os

# qianfan sdk 鉴权
os.environ["QIANFAN_AK"] = "your_ak"
os.environ["QIANFAN_SK"] = "your_sk"

首先定义两段prompt， 第一段Prompt主要指令为给出一个标题得到一份电影的主干，第二段为电影主干生成一篇评论

In [2]:
from langchain.prompts import PromptTemplate

synopsis_prompt = PromptTemplate.from_template("""你是一位编剧。鉴于电影的标题，你的任务是为该标题撰写一个电影剧情的主干。

标题：{title}
编剧：以下是上述电影的主干："""
)

review_template =  """你是《豆瓣影评》的深度用户。鉴于该电影的故事主干，你的任务是为该电影撰写一篇宣传称赞的评论。

主干：
{synopsis}
《豆瓣影评》戏剧评论家对上述戏剧的评论：:"""

review_prompt = PromptTemplate.from_template(review_template)

title="众里寻他千百度"

In [4]:
from qianfan.utils.logging import enable_log, TRACE_LEVEL
enable_log(TRACE_LEVEL)

## LCEL(推荐)

LCEL（LangChain Expression Language ）是Langchain对于组装多个Chain的一种语法设计，通过LCEL可以快速简单的串联一个多个变量，PromptTemplate和Chain的组合Pipeline，底层通过实现在Runnable中的__or__()实现该语法，
对于Langchain中的Prompt，LLM，Chain，Parser等各种组件来说，都派生自Runnable，所以可以通过以下的语法进行串联，以实现多个LLM+prompt的链式调用

In [5]:
from langchain.chat_models import QianfanChatEndpoint 
from langchain.schema import StrOutputParser

llm = QianfanChatEndpoint()
chain = (
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)
chain.invoke({"title": title})

[DEBUG][2024-07-22 16:23:22.513] openapi_requestor.py:398 [t:6235107328]: requesting llm api endpoint: /chat/eb-instant
[TRACE][2024-07-22 16:23:22.528] base.py:175 [t:6235107328]: raw request: QfRequest(method='POST', url='https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant', query={}, headers={'Request_id': 'sdk-py-0.4.3rc0-6zGTPwpYVTvFiBOD'}, json_body={'top_p': 0.8, 'temperature': 0.95, 'penalty_score': 1, 'stop': None, 'messages': [{'role': 'user', 'content': '你是一位编剧。鉴于电影的标题，你的任务是为该标题撰写一个电影剧情的主干。\n\n标题：众里寻他千百度\n编剧：以下是上述电影的主干：'}], 'stream': False, 'extra_parameters': {'request_source': 'qianfan_py_sdk_v0.4.3rc0'}}, files={}, retry_config=RetryConfig(retry_count=0, timeout=60, max_wait_interval=120, backoff_factor=1, jitter=1, retry_err_codes={2, 336100, 15, 18, 336501, 336502}))
[DEBUG][2024-07-22 16:23:30.325] openapi_requestor.py:398 [t:8344509248]: requesting llm api endpoint: /chat/eb-instant
[TRACE][2024-07-22 16:23:30.328] base.py:175 [t:8344509248]: 

'这是一部充满了人性和希望的现代都市电影，其主线围绕一个年轻的软件工程师李千寻的故事展开。在李千寻身上，我们看到了科技的魅力，同时也看到了他在面临困境时，如何坚持自己的理想，重新站起来面对生活。\n\n故事从李千寻的坚定和努力出发，一直延续到他遭遇的挫折和困惑。正是这些痛苦和迷茫，让他的故事更加真实和感人。在故事的转折点上，李千寻遇到了桃花，这个偶然的机会改变了他的生活。桃花的出现，不仅让李千寻重新找到了生活的方向，也让他重新找回了对理想的追求。\n\n电影的结尾部分，李千寻站在城市的高楼大厦之间，远望远方的风景，露出了满足和幸福的笑容。这一幕不仅让人感受到了他的成长和成功，也让人感受到了他对未来的信心和期待。\n\n总的来说，《豆瓣影评》对这部电影的评价非常高。它不仅是一部讲述科技和人生的电影，更是一部充满人性、温暖和希望的电影。它让我们看到了在困境中坚持理想的重要性，也让我们看到了在成长过程中，我们如何找到属于自己的新天地。这是一部值得一看的电影，我相信它会给你带来深刻的思考和启示。'

通过`RunnablePassthrough.assign`我们可以将将之前的输入字段进行透传，并分配我们本次的输出和字段名

In [30]:
from langchain.schema.runnable import RunnablePassthrough

synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = {"synopsis": synopsis_chain} | RunnablePassthrough.assign(custom_review_field=review_chain)
chain.invoke({"title": title})

{'synopsis': '电影《众里寻他千百度》的主干：\n\n故事发生在一个现代都市，讲述了一群年轻人在寻找自我和真爱的过程中，经历了种种挫折和成长的故事。\n\n故事的主角是一个名叫小杨的年轻人，他是一个性格内向、不善言辞的人，一直生活在自己的世界里。他有一个梦想，就是找到一个能够理解他、与他心灵相通的人。然而，在现实生活中，他很难找到这样的人，因此他决定通过互联网寻找他的另一半。\n\n小杨在网络上结识了一群志同道合的朋友，他们都是寻找真爱的年轻人。这群人在一起分享彼此的经历和感受，互相鼓励和支持。在这个过程中，小杨逐渐变得更加自信和勇敢，也开始尝试一些以前从未做过的事情，比如参加社交活动、改变自己的形象等等。\n\n然而，寻找真爱的过程并不总是一帆风顺。小杨和他的朋友们遇到了许多挫折和困难，比如被人误解、被人拒绝、甚至是网络诈骗等等。他们中的一些人在经历了这些挫折之后，选择了放弃寻找真爱，但小杨却并没有放弃。他坚信只有不断尝试和努力，才能找到真正的爱情。\n\n最后，小杨在一次偶然的机会下，遇到了一个和他有着相似经历的女孩小雨。他们在一场社交活动中相识，并且逐渐发现了彼此的共同点。小雨是一个善良、温柔、理解小杨的人，她让小杨感受到了真正的爱情和温暖。最终，他们走到了一起，并一起面对未来的挑战和机遇。\n\n这个故事的主干围绕着寻找真爱的主题展开，通过一群年轻人的经历和成长，展现了现代都市中年轻人面对爱情的态度和选择。同时，也表达了坚持和勇敢面对困难的重要性。电影的结尾温馨感人，让人感受到了爱情的力量和美好。',
 'custom_review_field': '《众里寻他千百度》是一部充满情感和感人的电影，它通过一群年轻人在寻找真爱的过程中所经历的种种挫折和成长，展现了现代都市中年轻人面对爱情的态度和选择。电影的主干清晰明了，故事情节紧凑，人物形象鲜明，情感表达真挚，让人深受感动。\n\n首先，电影中的主角小杨是一个内向、不善言辞的人，他一直生活在自己的世界里，寻找着真正的爱情。他的经历和成长过程非常真实，让人感到非常亲切。他的朋友们也是一群寻找真爱的年轻人，他们在一起分享彼此的经历和感受，互相鼓励和支持。这些人物形象都非常生动，让人感到非常真实和亲切。\n\n其次，电影中的情节非常感人。小杨和他的朋友们遇到了许多挫折和困难，比如被人误解、被人拒绝、甚至是

## SequentialChain

顺序链允许您连接多个链并将它们组合成执行某个特定场景的管道。有两种类型的顺序链：

- SimpleSequentialChain：最简单的SequentialChain形式，其中每个步骤都具有单一的输入/输出，一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用的SequentialChain形式，允许多个输入/输出。

### SimpleSequentialChain

In [8]:
from langchain.chains import LLMChain
from langchain.chat_models import QianfanChatEndpoint 
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = QianfanChatEndpoint(model="ERNIE-Bot", temperature=0.5)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt)

In [9]:
review_chain = LLMChain(llm=llm, prompt=review_prompt)

In [10]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

review = overall_chain.run(title)



> Entering new SimpleSequentialChain chain...
故事开始于一个繁华的城市，主人公李翔（男，30岁，职业：自由作家）坐在咖啡厅的角落，手中紧握着一封破旧的信。这封信引领他来到这个城市，寻找一个失散多年的女子——苏梦（女，28岁，职业：舞蹈教师）。

李翔与苏梦相识于十年前的一个夏天。当时，他们都在海边的小镇上度假。苏梦的美丽与才华给李翔留下了深刻的印象，他们度过了一个美好的夏天。然而，分别后他们失去了联系。这封信是李翔在整理旧物时发现的，信是苏梦写给他的，信中提到他们在那个夏天曾共同许下的诺言。

李翔决定寻找苏梦，他开始在城市的各个角落寻找她的踪迹。在这个过程中，他遇到了许多人和事，也影响了他对生活的看法。他发现，除了苏梦，他还有许多其他值得珍惜的人和事。

在寻找的过程中，李翔遇到了一个叫做林悦（女，25岁，职业：酒吧歌手）的女孩。林悦对生活充满热情，她鼓励李翔继续寻找苏梦，同时也启发了他去追寻自己的梦想。在林悦的陪伴下，李翔开始了一段充满挑战和希望的旅程。

经过一系列的挫折和困难，李翔终于找到了苏梦。然而，他发现苏梦已经有了自己的生活和家庭。李翔深感失落，但同时也明白了他在这段时间里所得到的成长和启示。他意识到，生命中有很多值得追求的东西，而这些东西并不一定要在一个人身上找到。

最后，李翔决定和林悦一起继续追求他们的梦想。他们在这个城市里开创了自己的事业，并且帮助了许多像他们一样有梦想的年轻人。他们的故事激励了更多的人去追求自己的梦想和生活价值。

电影以李翔和林悦在城市中的奋斗和成长为主线，展现了现代人在追求梦想和生活价值的过程中所经历的挫折、成长和启示。同时，电影也表达了珍惜身边的人和事，勇敢面对生活的态度。
《寻找苏梦》是一部感人至深的现代爱情故事，同时也是一部充满启示和鼓励的人生旅程。影片以李翔的寻找为主线，展现了一个男人在追寻自己内心深处的梦想和感情的过程中所经历的挫折、成长和启示。

影片的情节紧凑，人物形象鲜明。每个角色都有自己的故事和价值，他们的经历和感情相互交织，形成了一幅丰富多彩的人生画卷。特别是李翔和苏梦之间的感情纠葛，以及他们在寻找过程中所经历的种种挫折和磨难，让人不禁为之动容。

影片的拍摄手法也非常出色，导演通过运用独特的视角和拍摄技巧，将现代城市的繁华与人们内心的情感世界完美地结合

In [11]:
print(review)

《寻找苏梦》是一部感人至深的现代爱情故事，同时也是一部充满启示和鼓励的人生旅程。影片以李翔的寻找为主线，展现了一个男人在追寻自己内心深处的梦想和感情的过程中所经历的挫折、成长和启示。

影片的情节紧凑，人物形象鲜明。每个角色都有自己的故事和价值，他们的经历和感情相互交织，形成了一幅丰富多彩的人生画卷。特别是李翔和苏梦之间的感情纠葛，以及他们在寻找过程中所经历的种种挫折和磨难，让人不禁为之动容。

影片的拍摄手法也非常出色，导演通过运用独特的视角和拍摄技巧，将现代城市的繁华与人们内心的情感世界完美地结合在一起。每一个镜头都充满了情感和力量，让人对生活的美好有了更深刻的认识。

此外，影片还充满了许多细腻的情感元素。例如，李翔与林悦之间的友谊和相互扶持，以及他们在追求梦想的过程中所经历的成长和变化，都让人感受到了生活的美好和温暖。这些细腻的情感元素不仅让人对故事产生了共鸣，也让人对生活有了更深刻的认识和理解。

总的来说，《寻找苏梦》是一部充满情感和启示的电影。它不仅展现了一个男人在追寻自己内心深处的梦想和感情的过程中所经历的挫折、成长和启示，更是一部充满力量和温暖的电影。它鼓励人们勇敢面对生活中的挑战和困难，同时也提醒人们珍惜身边的人和事，去追求自己的梦想和生活价值。对于喜欢浪漫爱情故事的观众来说，《寻找苏梦》绝对是一部不容错过的佳作。


### SequentialChain

In [19]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
synopsis_template = """"你是一位编剧。鉴于电影的标题和背景，你的任务是为该标题撰写一个电影剧情的主干。

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)
synopsis_chain = LLMChain(
    llm=llm, prompt=synopsis_prompt_template, output_key="synopsis"
)

In [21]:
# This is an LLMChain to write a review of a play given a synopsis.
# template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

# Play Synopsis:
# {synopsis}
# Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

era = "21世纪20年代的广东深圳"
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)


overall_chain({"title": title, "era": era})



> Entering new SequentialChain chain...

> Finished chain.


{'title': '众里寻他千百度',
 'era': '21世纪20年代的广东深圳',
 'synopsis': '标题：众里寻他千百度\n\n21世纪20年代的广东深圳，一个繁华而充满活力的城市。李明，一个才华横溢的年轻人，为了实现自己的梦想，离开故乡来到深圳寻求发展。他渴望成为一名成功的企业家，以改变自己和家人的命运。\n\n在深圳的繁华都市中，李明结识了许多志同道合的朋友，他们一起为了梦想努力奋斗。然而，在追逐梦想的道路上，李明渐渐发现，现实往往比想象中更加残酷。\n\n一次偶然的机会，李明结识了一位神秘女子林燕。林燕拥有着与众不同的气质和优雅的举止，让李明为之倾心。然而，林燕似乎隐藏着一些不为人知的秘密。\n\n在林燕的引导下，李明开始了一段充满挑战和奇遇的冒险之旅。他逐渐发现，自己正置身于一个巨大的阴谋之中，而林燕则是他揭开真相的关键人物。\n\n在这场冒险中，李明不断地挑战自己的极限，成长为一个更加成熟、坚强的人。而林燕也逐渐展现出她真实的身份和目的，让李明深感震撼。\n\n最终，在一场惊心动魄的决战中，李明成功揭开了阴谋的真相，并实现了自己的梦想。而他与林燕之间的感情也经历了重重考验，最终修成正果。\n\n在这个充满挑战和奇遇的冒险之旅中，李明不断地寻找自己内心的力量和勇气，最终实现了自己的梦想。而这段旅程也让他的生命变得更加丰富多彩，让他明白了生命的真谛。',
 'review': '这部电影的故事主干充满了吸引力和张力，让人仿佛置身于一个充满挑战和奇遇的冒险之旅中。李明的成长和他与林燕之间的感情发展都让人深感震撼和感动。\n\n首先，电影的背景设定在21世纪20年代的广东深圳，一个繁华而充满活力的城市。这个背景为电影提供了一个非常具有现实感的舞台，让观众能够更好地理解和感受到李明在追逐梦想的过程中所面临的挑战和困难。\n\n其次，电影中的人物形象塑造非常成功。李明是一个充满活力和梦想的年轻人，他的决心和勇气让人感到敬佩。而林燕则是一个充满神秘和魅力的女性，她的出现让整个故事变得更加有趣和引人入胜。\n\n此外，电影的剧情设计也非常巧妙。在李明追求梦想的过程中，他不仅面临着来自外界的竞争和压力，还要面对自己内心的挣扎和矛盾。这些情节的设计让观众更加深入地了解了李明的内心世界和他的成长历程。\n\n最后，电影的制作质量也非常出色。无论是画面、音效还

### 使用Memory管理复杂的上下文

对于复杂场景下需要使用Memory管理上下文，Sequential可以实现更复杂的逻辑，以下例子包含了多步LLMChain的调用，同时存在前后输入输出依赖，并使用Memory管理了整个调用链上保持不变的变量例如`title`,`location`等

In [23]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

template = """你是一家电影公司的社交媒体经理。鉴于电影的标题、设定的时代、日期、时间和地点，以及电影的主干和评论，你的任务是为该电影撰写一则社交媒体帖子。

以下是关于电影的时间和地点的一些背景信息：:
时间: {time}
地点: {location}

电影主干:
{synopsis}
电影评论：
{review}

社交媒体帖子:
"""

time = "12月初冬，早7点"
location = "南山后海深圳湾公园"
prompt_template = PromptTemplate(
    input_variables=["synopsis", "review", "time", "location"], template=template
)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(
        memories={"time": time, "location": location}
    ),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True,
)

overall_chain({"title": title, "era": era})



> Entering new SequentialChain chain...

> Finished chain.


{'title': '众里寻他千百度',
 'era': '21世纪20年代的广东深圳',
 'time': '12月初冬，早7点',
 'location': '南山后海深圳湾公园',
 'social_post_text': '【电影推荐】《众里寻他千百度》—— 在深圳的早冬，感受爱与被爱的温度\n\n亲爱的朋友们，今天想给大家推荐一部充满情感与内涵的电影——《众里寻他千百度》。这部电影以21世纪20年代的广东深圳为背景，讲述了一段关于爱情、友情和人生价值的长篇画卷。\n\n故事的主角林小雨，是一个年轻有为的程序员，被指派开发一款名为"众里寻他千百度"的交友软件。这款软件运用了最先进的人工智能技术，旨在帮助用户在茫茫人海中寻找自己的另一半。然而，在全身心投入工作的同时，林小雨忽视了女友小月的感受。小月为了引起林小雨的注意，偷偷将交友软件中的异性匹配功能修改为同性匹配。当林小雨发现这个秘密时，他为了找出真相，尝试使用这款软件寻找答案。\n\n在这个寻找答案的过程中，林小雨遇到了形形色色的人，其中包括一位热情洋溢的同性恋者、一位寻找真爱的富家女、一位善良温暖的咖啡店老板娘等。他们都在使用"众里寻他千百度"这款软件，寻找自己的爱情和幸福。通过与这些人的交流，林小雨开始反思自己的生活和爱情。他渐渐明白，爱情不是一场游戏，而是一种责任和承诺。他决定放弃开发这款软件，转而投身公益事业，帮助那些需要帮助的人找到自己的幸福。\n\n在这个过程中，林小雨和小月重新找回了彼此的感情。他们一起参与公益事业，将爱传递给更多的人。最终，他们成为了一对幸福的恋人，过上了简单而美好的生活。虽然"众里寻他千百度"这款交友软件未能问世，但它却帮助了林小雨找到了自己的人生方向和爱情的真谛。\n\n这部电影的故事主干富有深度和内涵，展现了一段关于爱情、友情和人生价值的长篇画卷。影片以21世纪20年代的广东深圳为背景，描绘了现代都市中人们的生活状态和情感纠葛。林小雨这个角色塑造得十分立体。他是一个充满活力和创造力的程序员，但在忙碌的工作中却忽视了女友小月的感受。影片通过这个情节，深入探讨了现代都市中普遍存在的情感问题，即工作和生活的平衡。影片中的另一条主线是林小雨开发的交友软件"众里寻他千百度"。这个软件不仅具有先进的人工智能技术，还蕴含了人类对爱情和幸福的渴望。影片通过这个软件，展现了现代人在茫茫人